In [1]:
import numpy as np
import matplotlib.pyplot as plt

##mostra que ele cria uma cond inicial novamente para cada janela
import vtk
import matplotlib.pyplot as plt
import scipy  

import vtk
from vtk.util.numpy_support import vtk_to_numpy


def interpolate_2D( size, invar, *outvars):
        "Interpolate 2D outvar solutions onto a regular mesh"

        assert len(invar) == 2

        # define regular mesh to interpolate onto
        xs = [invar[k][:, 0] for k in invar]
        extent = (xs[0].min(), xs[0].max(), xs[1].min(), xs[1].max())
        xyi = np.meshgrid(
            np.linspace(extent[0], extent[1], size),
            np.linspace(extent[2], extent[3], size),
            indexing="ij",
        )

        # interpolate outvars onto mesh
        outvars_interp = []
        for outvar in outvars:
            outvar_interp = {}
            for k in outvar:
                outvar_interp[k] = scipy.interpolate.griddata(
                    (xs[0], xs[1]), outvar[k][:, 0], tuple(xyi)
                )
            outvars_interp.append(outvar_interp)

        return [extent] + outvars_interp

def all(invar, true_outvar, pred_outvar):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=pred_outvar["x1"])
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=true_outvar["x1"])
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=((true_outvar["x1"] -pred_outvar["x1"])**2)**0.5 ) 
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    
def call(invar, true_outvar, pred_outvar):
        extent, true_outvar, pred_outvar = interpolate_2D(
                200, invar, true_outvar, pred_outvar
            )
        ndim=2
        dims = list(invar.keys())
        fs = []
        for k in pred_outvar:
            f = plt.figure(figsize=(3 * 5, 4), dpi=100)
            for i, (o, tag) in enumerate(
                zip(
                    [true_outvar[k], pred_outvar[k], ((true_outvar[k] - pred_outvar[k])**2)**(0.5) ],
                    ["true", "pred", "diff"],
                )
            ):
                if (tag=="diff"):
                    to=o
                    to[0][0]=0
                plt.subplot(1, 3, 1 + i)
                if ndim == 1:
                    plt.plot(invar[dims[0]][:, 0], o[:, 0])
                    plt.xlabel(dims[0])
                elif ndim == 2:
                    plt.imshow(o.T, origin="lower", extent=extent)
                    
                    plt.xlabel(dims[0])
                    plt.ylabel(dims[1])
                    if(tag=="diff"):
                       
                        plt.clim(0,0.4)
                    else :
                        plt.clim(-.2,1)
                    plt.colorbar()
                plt.title(f"{k}_{tag}")
            plt.tight_layout()


def read_vtp(path):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(path)
    reader.Update()
    data = reader.GetOutput().GetPointData()
   # print(data)
    field_count = data.GetNumberOfArrays()
    return {data.GetArrayName(i): vtk_to_numpy(data.GetArray(i)) for i in range(field_count)}

def r(path):
    
    data=read_vtp(path)
    #print(data["t"])
    return data["t"],data["pred_x1"],data["true_x1"],data["true_w"],data["pred_w"]
def plot(path,legend=False):

    data=read_vtp(path)
    print(data)


   
    print(data)   

    
    X=data["pred_x1"]
    TX=data["true_x1"]
    
   # plt.plot(data['t'],((X-TX)**2)**0.5  ,"red",label=" Error U")
    
    X=data["pred_w"]
    TX=data["true_w"]
    #plt.plot(data['t'],((X-TX)**2)**0.5  ,"red",label=" Error W")
    
    plt.ylim(-1,1)
    if(legend==True):
        plt.legend(loc="best")
        plt.ylabel('U,W')
        plt.xlabel('T')
       
    
    
def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]



p=lambda s:'outputs/fhn0P/window'+str(s)+'/validators/validator.vtp'
#plot('outputs/fhn2eqv2/initial_conditions/validators/validator.vtp',True)


t,x,tx,w,tw= r('outputs/fhn0P/initial_conditions/validators/validator.vtp')
n_w=10

n=np.shape(x)[0]
T=np.zeros(n*n_w)
X=np.zeros(n*n_w)
W=np.zeros(n*n_w)
TW=np.zeros(n*n_w)
TX=np.zeros(n*n_w)

T[0:n]=t
X[0:n]=x
TX[0:n]=tx
W[0:n]=w
TW[0:n]=tw

for i in range(1,10):
    print(i)
    d=r(p(i))
    print(np.shape(d))
    T[n*i:n*(i+1)]=d[0]
    X[n*i:n*(i+1)]=d[1]
    TX[n*i:n*(i+1)]=d[2]
    W[n*i:n*(i+1)]=d[3]
    TW[n*i:n*(i+1)]=d[4]


print("max err",np.max(   (((X-TX)**2)**0.5 ))   )
print("mean err",np.mean( (((X-TX)**2)**0.5 ))   )

e=lambda x:np.expand_dims(x,axis=1)

invar={"t":e(T.T )   }
out={"x1":e(X), "w":e(W.T)      }
out_t={"x1":e(TX.T),"w":e(W.T)         }

plt.plot(e(T.T ),e(W.T),'-',label=" Pred W")

plt.plot(e(T.T ),e(X),'-',label="Pred U")


plt.plot(e(T.T ),e(TX.T),"--",color="grey",label=" True U")

    
plt.plot(e(T.T ),e(TW),"--",color="grey",label=" True W")

plt.ylim(-.2,1)
if(True):
        plt.legend(loc="best")
        plt.ylabel('U,W')
        plt.xlabel('T')


plt.savefig("fig0.png")

2023-06-20 21:50:22.412 (   0.300s) [         DA96740]       vtkXMLReader.cxx:306    ERR| vtkXMLPolyDataReader (0x560399b04650): Error opening file outputs/fhn0P/initial_conditions/validators/validator.vtp
2023-06-20 21:50:22.412 (   0.300s) [         DA96740]       vtkExecutive.cxx:740    ERR| vtkCompositeDataPipeline (0x560399b1a210): Algorithm vtkXMLPolyDataReader (0x560399b04650) returned failure for request: vtkInformation (0x560399b15580)
  Debug: Off
  Modified Time: 158
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0




KeyError: 't'

In [21]:
import numpy as np
import matplotlib.pyplot as plt

##mostra que ele cria uma cond inicial novamente para cada janela
import vtk
import matplotlib.pyplot as plt
import scipy  

import vtk
from vtk.util.numpy_support import vtk_to_numpy


def interpolate_2D( size, invar, *outvars):
        "Interpolate 2D outvar solutions onto a regular mesh"

        assert len(invar) == 2

        # define regular mesh to interpolate onto
        xs = [invar[k][:, 0] for k in invar]
        extent = (xs[0].min(), xs[0].max(), xs[1].min(), xs[1].max())
        xyi = np.meshgrid(
            np.linspace(extent[0], extent[1], size),
            np.linspace(extent[2], extent[3], size),
            indexing="ij",
        )

        # interpolate outvars onto mesh
        outvars_interp = []
        for outvar in outvars:
            outvar_interp = {}
            for k in outvar:
                outvar_interp[k] = scipy.interpolate.griddata(
                    (xs[0], xs[1]), outvar[k][:, 0], tuple(xyi)
                )
            outvars_interp.append(outvar_interp)

        return [extent] + outvars_interp

def all(invar, true_outvar, pred_outvar):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=pred_outvar["x1"])
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=true_outvar["x1"])
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    img = ax.scatter(invar["K"], invar["V"],invar["t"],c=((true_outvar["x1"] -pred_outvar["x1"])**2)**0.5 ) 
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()

    fig.colorbar(img)
    plt.show()
    
def call(invar, true_outvar, pred_outvar):
        extent, true_outvar, pred_outvar = interpolate_2D(
                200, invar, true_outvar, pred_outvar
            )
        ndim=2
        dims = list(invar.keys())
        fs = []
        for k in pred_outvar:
            f = plt.figure(figsize=(3 * 5, 4), dpi=100)
            for i, (o, tag) in enumerate(
                zip(
                    [true_outvar[k], pred_outvar[k], ((true_outvar[k] - pred_outvar[k])**2)**(0.5) ],
                    ["true", "pred", "diff"],
                )
            ):
                if (tag=="diff"):
                    to=o
                    to[0][0]=0
                plt.subplot(1, 3, 1 + i)
                if ndim == 1:
                    plt.plot(invar[dims[0]][:, 0], o[:, 0])
                    plt.xlabel(dims[0])
                elif ndim == 2:
                    plt.imshow(o.T, origin="lower", extent=extent)
                    
                    plt.xlabel(dims[0])
                    plt.ylabel(dims[1])
                    if(tag=="diff"):
                       
                        plt.clim(0,0.4)
                    else :
                        plt.clim(-.2,1)
                    plt.colorbar()
                plt.title(f"{k}_{tag}")
            plt.tight_layout()


def read_vtp(path):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(path)
    reader.Update()
    data = reader.GetOutput().GetPointData()
   # print(data)
    field_count = data.GetNumberOfArrays()
    return {data.GetArrayName(i): vtk_to_numpy(data.GetArray(i)) for i in range(field_count)}

def r(path):
    
    data=read_vtp(path)
    #print(data["t"])
    return data["t"],data["x1"],data["w"]

def plot(path,legend=False):

    data=read_vtp(path)
    print(data)


   
    print(data)   

    
    X=data["pred_x1"]
    TX=data["true_x1"]
    
   # plt.plot(data['t'],((X-TX)**2)**0.5  ,"red",label=" Error U")
    
    X=data["pred_w"]
    TX=data["true_w"]
    #plt.plot(data['t'],((X-TX)**2)**0.5  ,"red",label=" Error W")
    
    plt.ylim(-1,1)
    if(legend==True):
        plt.legend(loc="best")
        plt.ylabel('U,W')
        plt.xlabel('T')
       
    
    
def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]



p=lambda s:'outputs/fhn0P/window'+str(s)+'/inferencers/inferencer.vtp'
#plot('outputs/fhn2eqv2/initial_conditions/validators/validator.vtp',True)


t,x,w= r('outputs/fhn0P/initial_conditions/inferencers/inferencer.vtp')
n_w=10

n=np.shape(x)[0]
print(n)
T=np.zeros(n*n_w)
X=np.zeros(n*n_w)
W=np.zeros(n*n_w)

T[0:n]=t
X[0:n]=x
W[0:n]=w

for i in range(1,10):
    #print(i)
    d=r(p(i))
    print(np.shape(d))
    T[n*i:n*(i+1)]=d[0]+i
    X[n*i:n*(i+1)]=d[1]
    W[n*i:n*(i+1)]=d[2]




e=lambda x:np.expand_dims(x,axis=1)

invar={"t":e(T.T )   }
out={"x1":e(X), "w":e(W.T)      }




plt.plot(T,W,'-',label=" Pred W")

plt.plot(T,X,'-',label="Pred U")


plt.ylim(-.2,1)
if(True):
        plt.legend(loc="best")
        plt.ylabel('U,W')
        plt.xlabel('T')


plt.savefig("fig0.png")

6
(3, 5)


ValueError: could not broadcast input array from shape (5,) into shape (6,)

In [65]:

t,x,w= r('outputs/fhn0P/initial_conditions/inferencers/inferencer.vtp')
n_w=10

n=np.shape(x)[0]
print(x)

[ 0.00000000e+00 -1.28569377e-06 -2.56973203e-06 -3.85211752e-06
 -5.13285113e-06 -6.41192401e-06 -7.68934387e-06 -8.96513575e-06
 -1.02392696e-05 -1.15117382e-05 -1.27825888e-05 -1.40517823e-05
 -1.53193214e-05 -1.65852307e-05 -1.78495193e-05 -1.91121399e-05
 -2.03731288e-05 -2.16324952e-05 -2.28902081e-05 -2.41463003e-05
 -2.54007409e-05 -2.66536026e-05 -2.79047526e-05 -2.91543274e-05
 -3.04022797e-05 -3.16485894e-05 -3.28932983e-05 -3.41363484e-05
 -3.53777868e-05 -3.66176355e-05 -3.78558725e-05 -3.90924470e-05
 -4.03274353e-05 -4.15608229e-05 -4.27925734e-05 -4.40227013e-05
 -4.52513013e-05 -4.64782825e-05 -4.77036119e-05 -4.89273589e-05
 -5.01495088e-05 -5.13700070e-05 -5.25889918e-05 -5.38062886e-05
 -5.50221594e-05 -5.62362657e-05 -5.74488513e-05 -5.86598617e-05
 -5.98692422e-05 -6.10771895e-05 -6.22833832e-05 -6.34879543e-05
 -6.46910194e-05 -6.58925928e-05 -6.70925219e-05 -6.82909013e-05
 -6.94877162e-05 -7.06829087e-05 -7.18766532e-05 -7.30687389e-05
 -7.42592674e-05 -7.54482

In [19]:
import os
os.chdir('0P')

FileNotFoundError: [Errno 2] No such file or directory: '0P'